## 기본적인 Prompt 구조 이해
prompt에는 3가지 종류의 역할(role)이 존재

1. sysytem prompt : 사용자 prompt를 입력받기 이전에 정의되는 전제 및규칙
2. user prompt : 사용자가 LLM에게 실제로 요청하는 prompt
3. assistant prompt : LLM이 응답하는 prompt

#### system prompt란?
user prompt를 LLM에게 전달하기 전, 관련된 맥락이나 응답지침 등을 설정하는 prompt
- 출력 형태 지정(ex. JSON 딕셔너리 등)
- 페르소나(투자 전문가, 예술가 등) 및 어조 설정(공손한, 전문적인 등)
- 모델이 지켜야할 규칙들 설정
- 기타 base가 되는 외부 정보 및 지식 주입


#### ChatGPT를 포함한 웬만하면 LLM모델 서비스들은 prompt 입력시 기본적으로 지정된 system prompt가 내부에서 동작하고 있음

In [2]:
from openai import OpenAI
from getpass import getpass

In [4]:
MY_API_KEY = getpass.getpass("input your key")

input your key ········


In [6]:
client = OpenAI(api_key = MY_API_KEY)

completion  = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                             messages = [{'role' : 'system',
                                                          'content' : '당신은 물리학 선생님입니다. 초등학생에게 설명하듯 아주 쉽고 친근하게 설명해야 합니다.'},
                                                         {'role' : 'user',
                                                          'content': '왜 하늘은 하늘색인가요?'}],
                                             temperature = 0
                                            )

print(completion.choices[0].message.content)

하늘은 왜 하늘색인지 궁금하시군요! 하늘은 우리가 보는 것과 같이 파란색으로 보이는데요, 이것은 태양빛이 하늘과 대기에 닿았을 때 일어나는 현상 때문이에요.

태양빛은 여러 색으로 이루어져 있는데, 그 중에서 파란색 빛은 다른 색보다 더 짧은 파장을 가지고 있어요. 그래서 하늘과 대기 속의 먼지와 수증기 등이 파란색 빛을 더 많이 흡수하고 흩뿌리게 되는데요. 이런 현상 때문에 우리가 보는 하늘은 파란색으로 보이게 되는거죠!

그래서 하늘은 하늘색인 것이에요. 파란색 빛이 더 많이 흩뿌려져서 우리 눈에 파란색으로 보이는 거죠. 재미있는 현상이죠!


- 텍스트가 짧다면 user prompt에 system prompt 내용을 같이 작성해도 무방함

### Stream 객체 살펴보기
- stream = True로 지정시 GPT가 문장을 모두 완성하기 전에 각 토큰별로 완성되는대로 바로바로 보여줌

In [16]:
# stream=True로 설정하게 되면 응답 객체가 ChatCompletion이 아니라 ChatCompletionChunk로 변환됨
 # Chunk는 토큰들의 의미있는 집합 
completion_stream  = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                                    messages = [{'role' : 'user',
                                                                 'content': '왜 하늘은 하늘색인가요?'}],
                                                    temperature = 0,
                                                    stream=True
                                                   )

for i in completion_stream:
    # print(i)
    content = i.choices[0].delta.content

    # content가 비어있지 않은 경우 
    if content is not None:
        print(content, end = '')

하늘은 하늘색인 이유는 대기 중의 분자들이 푸른 빛을 흡수하고 다른 색상의 빛을 반사하기 때문입니다. 태양 빛은 다양한 색상의 빛으로 구성되어 있지만, 대기 중의 분자들이 푸른 빛을 흡수하고 다른 색상의 빛을 반사하게 되어 우리 눈에는 하늘이 푸르게 보이는 것입니다. 이러한 현상을 산란이라고 하며, 이로 인해 하늘은 하늘색으로 보이게 됩니다.

### LLM의 task에 따른 평가지표들을 알아보고 대화 및 Q&A task에 실제로 적용해보자
#### 전통적인 Language Model 평가 지표
1. MMLU(Massive Multitask Language Understanding)
 - 다양한 분야에 대한 질문 후 정답을 찾아내게 하는 객관식 시험
2. HellaSwag
 - 문장들을 주고 이어지는 마지막 문장으로 가장 적합한 문장들 4개중 하나를 고르는 시험

#### 위 평가지표들은 범용 Language Model에 대한 평가 방법론들이라 현재 우리가 진행할 대화 및 Q&A에는 적합하지 않음

### 대화 및 Q&A task에 적합한 평가 방식
1. **Human Based Evaluation** - 사람이 직접 평가하는 방법
2. **Model Based Evaluation** - LLM이 평가하는 방법
3. **Code Based Evaluation** - 코드로 평가하는 방법

### 1) Human Based Evaluation 
- 전문가 블라인드 A/B테스트(각기 다른 LLM의 2가지 답변 중 더 좋은 답변을 사람이 선택)
- 명확한 결과로 성능을 판단하기 쉬움
- 많은 인력에 따른 비용과 시간이 필요함

#### LMSys사 Chatbot Arena 평가방법
- 대표적인 Human Based 방식으로 동일한 질문에 대해 2개의 모델의 답변을 보고 승/패/무 투표 이후 모델명을 공개하는 방식
- 사이트 : https://chat.lmsys.org/

### 2) Model Based Evaluation
고성능 LLM(GPT-4o이상)을 통해 평가하는 방법
- 실제로 사람이 평가하는 것과 굉장히 유사하다는 논물 결과들이 나오고 있음

평가하는 방식에는 3가지가 존재
1. **Pairwise Comparison**
-  2개의 평가받을 모델에 같은 질문을 하고, 고성능 모델이 2개의 답변을 받아 둘 중 어떤 답변이 더 좋은지 또는 무승부인지를 출력
2. **Single Answer Grading**
- 질문과 답변이 있을 때 모델이 답변에 점수를 매기는 것
3. **Reference-Guided Grading**
- 예시 답변을 주고 이와 비교하여 +,-로 상대적인 점수를 매기는 방식

### 3) Code Based Evaluation
우리에게 익숙한 코드/로직을 통한 평가 방법
- Accuray, Precision, Recall, F1-Score
- ROUGE(Recall-Oriented Understudy for Gisting Evaluation) : 자연어 생성 및 요약을 평가
- BLEU(Bilingual Evaluation Understudy) : 번역, 요약, 자연어 생성 등을 평가

단, Human based 및 Model based에 비해 실제 사용자들의 만족과는 다소 거리가 있을 수 있음

### 최신 논문을 참조하여 대화 및 Q&A task에 적합한 평가 방법에 대한 힌트를 얻어보자
- 논문명 : Judging LLM-as-a-Judge with MT-Bench and ChatbotArena
- GPT-4로 평가 진행(그때 당시 가장 성능이 좋은 LLM)
- 해당 논문의 핵심은 LLM의 평가 결과가 사람의 평가 결과와 일치율이 점점 높아지고 있다는 것
- 이는 기계의 판단이 인간의 판단에 도움이 되며, 기계를 통한 자동화된 평가 프로세스 개발이 가능하다는 것을 시사함
- 실제로 해당 논문에서 인간의 75%가 기계의 판단이 합리적이라 생각했고 34%가 자신의 선택을 바꿀 의향이 있다고 응답함

### benchmark : 다양한 시스템, 모델, 소프트웨어 등을 평가하고 비교하기 위해 사용되는 표준 테스트 세트 또는 기준

### MT-bench(Multi-turn benh)
- multi-turn(이어지는 대화) 능력을 평가하는 벤치마크
- 먼저 58명의 전문가 모델의 응답을 평가하고 LLM을 심판으로 사용하여 사람의 평가와 일치하는지 검증하는 방식
- ※ multi-turn 대화 : MT-bench에는 8개 카테고리(작문, 역할놀이, 추출, 추론, 수학, 코딩, 지식1(물리/수학), 지식2(인문/사회))의 80개의 고품질 질문으로 구성되어 있고, 각 질문은 여러차례의 응답을 요구하여 모델의 대화 지속능력을 평가함

In [29]:
question = '하늘은 왜 하늘색인가요?'

In [33]:
completion = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                            messages = [{'role' : 'user',
                                                         'content' : question}],
                                            temperature = 0
                                           )
answer_a = completion.choices[0].message.content
print(answer_a)

하늘은 파란색으로 보이는데, 이는 태양 빛이 대기 중의 공기 분자들과 상호 작용하여 발생하는 현상입니다. 태양 빛은 다양한 색상의 빛으로 구성되어 있지만, 대기 중의 공기 분자들이 파란색 빛을 더 많이 흡수하고 다른 색상의 빛을 덜 흡수하기 때문에 하늘이 파란색으로 보이는 것입니다. 이러한 현상을 산란이라고 하며, 이로 인해 하늘은 파란색으로 보이게 됩니다.


In [39]:
completion = client.chat.completions.create(model = 'gpt-4-turbo',
                                            messages = [{'role' : 'user',
                                                         'content' : question}],
                                            temperature = 0
                                           )
answer_b = completion.choices[0].message.content
print(answer_b)

하늘의 색깔이 하늘색, 즉 파란색으로 보이는 이유는 대기 중의 분자들과 작은 입자들이 태양 빛을 산란시키기 때문입니다. 이 현상을 '레일리 산란'이라고 합니다.

태양에서 오는 빛은 여러 가지 색의 빛으로 구성되어 있으며, 이 빛들은 파장의 길이에 따라 다르게 산란됩니다. 레일리 산란은 파장이 짧은 빛 (예: 파란색, 보라색)이 파장이 긴 빛 (예: 빨간색, 주황색)보다 더 많이 산란되는 현상입니다. 파란색 빛의 파장은 매우 짧기 때문에 대기 중을 통과할 때 다른 색의 빛보다 훨씬 더 많이 산란되어, 우리 눈에는 하늘이 파란색으로 보이게 됩니다.

그러나 해가 지평선에 가까울 때, 즉 일출이나 일몰 때에는 하늘이 빨강, 주황, 노랑과 같은 따뜻한 색으로 보이는데, 이는 태양 빛이 대기를 통과하는 거리가 훨씬 길어져서 파란색과 보라색 빛이 대기에 의해 더 많이 산란되고, 빨간색과 주황색 빛이 상대적으로 덜 산란되어 우리 눈에 도달하기 때문입니다.


In [40]:
# 논문 프롬프트 참고 (a와 b중 더 좋은 답변을 찾기 위한 프롬프트)
prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Question]
{question}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
print(prompt)

[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Question]
하늘은 왜 하늘색인가요?
[The Start of Assistan

In [43]:
# 가장 최신 gpt-4o 모델로 앞선 두 모델의 응답을 평가
completion = client.chat.completions.create(model='gpt-4o',
                                            messages = [{'role' : 'user',
                                                         'content' : prompt}],
                                            temperature = 0,
                                           )
print(completion.choices[0].message.content)

Both Assistant A and Assistant B provide accurate explanations for why the sky appears blue, focusing on the concept of Rayleigh scattering. They both mention that shorter wavelengths of light, such as blue, are scattered more than longer wavelengths, which is why the sky appears blue during the day. They also both explain why the sky appears red or orange during sunrise and sunset, due to the longer path through the atmosphere and the scattering of shorter wavelengths.

However, Assistant A provides a slightly more detailed explanation by mentioning "Mie scattering" in the context of the sky's color during sunrise and sunset, although this is not entirely accurate as Mie scattering is not the primary reason for the color change at these times. Assistant A also provides a more structured explanation, starting with the interaction of sunlight with atmospheric molecules and then explaining the scattering phenomena.

Assistant B's response is concise and accurate but lacks the additional 

### 장단점 비교
1. Human Based Evaluation
- 통제된 환경을 가정했을 때 사람이 직접 평가한 방법이라 안정적이고 신뢰할 수 있음
- 전문가가 아닌 불특정 다수의 경우 약간의 노이즈가 발생할 수 있음
- 전문가가 아닌 경우 평가 정확도 저하 및 속도가 더 오래걸릴 수 있음

2. Model Based Evaluation
- 사람 평가와 어느정도 유사한 수준의 평가를 내릴 수 있음
- 평가를 위해 API호출이 필요한데 평가 데이터가 굉장히 많을 경우 수백만원 이상 소요됨

3. Code Based Evaluation
- 위 방법들에 비해 인력 비용, 모델 호출 비용 등이 없는 무료 평가 방법
- task에 따라 활용할 수 있는 범위가 제한적
- 사람들에게 적합한 답변을 선택하는데 있어서는 신뢰도가 상대적으로 떨어지는 편

### 프롬프트 엔지니어링 고급 기법 적용
1. **few-shot**
- 참고할 수 있는 문제-정답 예시나 사례들을 프롬프트에 추가하여 질의
- 논문 : https://arxiv.org/abs/2005.14165
2. **Chain-of-thought(CoT)**
- Few-shot에 추가로 문제 해결과정을 단계별로 모델에게 알려주면서 질의
- 논문 : https://arxiv.org/abs/2201.11903

#### Zero-show
- 질의에 아무런 예시가 없는 경우

In [55]:
prompt = """Q: Who wrote the book 'HARRY POTTER'?"""

completion = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                            messages = [{'role' : 'user',
                                                         'content' : prompt}],
                                            temperature = 0
                                           )
print(completion.choices[0].message.content)

J.K. Rowling wrote the book 'Harry Potter'.


#### Few-show
- few-show(One-shot)
  

In [60]:
# 예시를 명시해주니 이름만 말함
prompt = """ Answer these question:
Q: Who wrote the book 'HARRY POTTER'?

Below is an example for your reference.
Q: Who sang 'One Call Away'?
A: Charlie Puth

"""

completion = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                            messages = [{'role' : 'user',
                                                         'content' : prompt}],
                                            temperature = 0
                                           )
print(completion.choices[0].message.content)

A: J.K. Rowling


#### Chain of Thought
- 마이크로 소프트에서 사용한 예시 프롬프트 활용

In [67]:
prompt = """Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, \n
how many apples does Alice have?"""

completion = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                            messages = [{'role' : 'user',
                                                         'content' : prompt}],
                                            temperature = 0
                                           )

print(completion.choices[0].message.content)

Alice would have 5 apples - 3 thrown + 2 given to Bob + 1 given back by Bob = 5 apples. 

So, Alice still has 5 apples.


#### CoT + One-shot
- Below ~ : One-shot
- 계산식 : CoT

In [71]:
prompt = """Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, \n
how many apples does Alice have?

    Below is an example for your reference.

    Lisa has 7 apples, throws 1 apple, gives 4 apples to Bart and Bart gives one back:
    7 - 1 = 6
    6 - 4 = 2
    2 + 1 = 3


"""

completion = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                            messages = [{'role' : 'user',
                                                         'content' : prompt}],
                                            temperature = 0
                                           )

print(completion.choices[0].message.content)

Alice has 5 apples - 3 thrown = 2 apples
2 apples - 2 given to Bob = 0 apples
0 apples + 1 from Bob = 1 apple

Therefore, Alice has 1 apple.


#### 또 다른 프롬프트 고도화 예시
- KMMLU(Measuring Massive Multitask Language Understanding in Korea) 논문의 프롬프트 사용

In [74]:
# 실제 정답은 B=6
question = 'x, y가 세 부등식 y ≤ x+3, y ≤ -4x+3, y ≥ 0을 만족할 때, x+y의 최댓값을 M, 최솟값을 m이라 하면 M-m의 값은?'
A = 4
B = 6
C = 8
D = 10

In [94]:
prompt = f"""{question}
A. {A}
B. {B}
C. {C}
D. {D}
정답 : 
"""

In [96]:
completion = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                            messages = [{'role' : 'user',
                                                         'content' : prompt}],
                                            temperature = 0
                                           )

print(completion.choices[0].message.content)

C. 8


#### 3.5모델 고도화 시키기
- 페르소나 적용
- 영문 prompt 작성
- 효과적인 prompt 작성

In [105]:
# 페르소나 적용
prompt = f"""You are an Professinal in Mathematics.
Bellow is given a math question in Korea.

{question}
A. {A}
B. {B}
C. {C}
D. {D}
정답 : 
"""

completion = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                            messages = [{'role' : 'user',
                                                         'content' : prompt}],
                                            temperature = 0
                                           )

print(completion.choices[0].message.content)

C. 8


In [147]:
# 페르소나 적용
prompt = f"""You are an Professinal in Mathematics.
Bellow is given a math question in Korea.
You have to think carefully and step by step about the question and choose one of four given answers.
Only one of them is ture. and explain it in Korea.
After calculating, check the result again.

Give resason step by step and carefully about why you think your answer is correct.


{question}
A. {A}
B. {B}
C. {C}
D. {D}
정답 : 
"""

completion = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                            messages = [{'role' : 'user',
                                                         'content' : prompt}],
                                            temperature = 0
                                           )

print(completion.choices[0].message.content)

C. 8

우선 주어진 부등식을 그래프로 그려보겠습니다.

1. y ≤ x+3
이는 y = x+3인 직선을 포함하며, (0,3)을 지나고 기울기가 1인 직선입니다.

2. y ≤ -4x+3
이는 y = -4x+3인 직선을 포함하며, (0,3)을 지나고 기울기가 -4인 직선입니다.

3. y ≥ 0
이는 x축 위의 점들을 포함하는 영역입니다.

이 세 부등식을 만족하는 영역은 y ≤ x+3, y ≤ -4x+3, y ≥ 0이 교차하는 부분으로, 삼각형 모양의 영역입니다.

이제 x+y의 최댓값을 구하기 위해, 삼각형의 꼭지점을 찾아야 합니다. 이 꼭지점은 세 부등식이 만나는 지점 중에서 가장 큰 값을 가지게 됩니다.

y = x+3과 y = -4x+3을 동시에 만족하는 지점을 찾으면, x = 0, y = 3이 됩니다. 따라서 이 꼭지점은 (0,3)입니다.

따라서 x+y의 최댓값 M은 3+0 = 3이 됩니다.

이제 x+y의 최솟값을 구하기 위해, 삼각형의 다른 꼭지점을 찾아야 합니다. 이 꼭지점은 세 부등식이 만나는 지점 중에서 가장 작은 값을 가지게 됩니다.

y = x+3과 y = 0을 동시에 만족하는 지점을 찾으면, x = -3, y = 0이 됩니다. 따라서 이 꼭지점은 (-3,0)입니다.

따라서 x+y의 최솟값 m은 0+(-3) = -3이 됩니다.

따라서 M-m = 3 - (-3) = 6이므로, 정답은 8이 됩니다.


### 프롬프트 엔지니어링 특징 정리
- 추가 학습이 없음에도 성능 개선의 가능성이 있기 때문에 가성비가 굉장히 좋음
- 더 좋은 모델을 사용하면 프롬프트 엔지니어링 없이도 해결할 수 있지만 비용 측면을 무시할 수 없기 때문에 먼저 프롬프트 엔지니어링으로 성능향상을 시도해보는 것이 좋음(특히 현업에서는 비용 문제에 굉장히 민감함)
- 단 모델의 알고리즘이 완전히 바뀌어 버리면 기존에 사용했던 방법들이 크게 달라질 수 있어서 변동성이 높은 편
- 프롬프트 엔지니어링으로도 해결이 되지 않으면 RAG나 Finetuning으로 성능을 향상시킬 수 있음